In [1]:
#IMPORT LIBRARIES
#Try to model blackjack outcomes long term
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#RANDOMIZE DRAWING CARDS
test =[]
card_options = [2,3,4,5,6,7,8,9,10,10,10,10,11]
for deals in range(1000):
    deals = random.choice(card_options)
    test.append(deals)


In [3]:
#VERIFY CARD PROBABILITIES ARE CORRECT

results = pd.DataFrame(test, columns = ["Card Values"])
#results["Card Values"].value_counts().plot(kind="bar")

In [4]:
#DEFINE ALL FUNCTIONS
def hit_me(current_total, player = "True"): #I use this function to model dealer hand too and I don't want hit_count to go up
    global thrid_card_list
    global forth_card_list
    global fifth_card_list
    global sixth_card_list
    
    another_card = random.choice(card_options)
    if another_card == 11 and current_total >=11: #ace only worth 1 if total goes over 21 (current total = 11)
        another_card = 1
    else:
        another_card = another_card
    global hit_count #have to define this as global or else these variables or only defined within the function. I want the rest of my code to use it also
    if player == "True":
        hit_count = hit_count + 1

    if hit_count == 1: #for analysis purposes to record additional cards drew
        thrid_card_list.append(another_card)
    elif hit_count == 2:
        forth_card_list.append(another_card)
    elif hit_count == 3:
        fifth_card_list.append(another_card)
    elif hit_count == 4:
        sixth_card_list.append(another_card)

    current_total = current_total + another_card
    return current_total

#define stay function
def stay(current_total):
    global hit
    global soft
    hit = False
    soft = False
    return current_total
    
#can't double down if already taken a card. Can have either (double or hit) or (double or stay). Default is to hit if cannot double
def double_down(current_total, Ds = "Hit"):
    global hit_count
    global bet
    global hit
    global soft
    global double
    if hit_count >= 1 and Ds == "Hit": #hit
        current_total = hit_me(current_total)
    elif hit_count >= 1 and Ds == "Stay": #stay
        current_total = stay(current_total)
    else: #double
        bet = 2*bet
        current_total = hit_me(current_total)
        hit = False
        soft = False
        double = True
    return current_total

#compare player vs dealer's hand
def compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card):
    global current_money
    global bet
    global blackjack

    #player bust
    if player_total > 21:
        current_money = current_money - bet
        outcome = "Bust"
    #blackjack
    elif (first_card + second_card) == 21:
        if (dealer_first_card + dealer_second_card) == 21:
            current_money = current_money
            outcome = "Push"
        else:
            current_money = current_money + blackjack*bet
            outcome = "Blackjack"
    #dealer bust
    elif dealer_total > 21:
        current_money = current_money + bet
        outcome = "Dealer Bust"
    #win
    elif player_total > dealer_total:
        current_money = current_money + bet
        outcome = "Win"
    #loss
    elif player_total < dealer_total:
        current_money = current_money - bet
        outcome = "Loss"
    #push
    elif player_total == dealer_total:
        current_money = current_money
        outcome = "Push"
    else:
        outcome = "Undefined"

    return [current_money, outcome]
 
def soft_totals(current_total, dealer_first_card):
    global hit
    global soft
    global hit_count
    while soft == True:                
        #if soft total >= 19, stay
        if (current_total >= 19):
            current_total = stay(current_total)
        #if soft total = 18 and dealer has <=6, double
        elif (current_total == 18) and dealer_first_card <= 6:
            current_total = double_down(current_total, Ds="Stay")
            #print(current_total, first_card, second_card, dealer_first_card)
        #if soft total = 18 and dealer has 7 or 8, stay
        elif (current_total == 18) and (dealer_first_card == 7 or dealer_first_card == 8):
            current_total = stay(current_total)
        #if soft total = 18 and dealer has >= 9, hit
        elif (current_total == 18) and (dealer_first_card >= 9):
            current_total = hit_me(current_total)
        #if soft total = 17 and dealer has 3 - 6, double
        elif (current_total == 17) and (dealer_first_card >= 3 and dealer_first_card <= 6):
            current_total = double_down(current_total)
        #if soft total = 17 and dealer has 2 or >=7, hit
        elif (current_total == 17) and (dealer_first_card == 2 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if soft total = 15-16 and dealer has 4 - 6, double
        elif (current_total == 15 or current_total == 16) and (dealer_first_card >= 4 and dealer_first_card <= 6):
            current_total = double_down(current_total)
        #if soft total = 15-16 and dealer has 2 or >= 7, hit
        elif (current_total == 15 or current_total == 16) and (dealer_first_card <= 3 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if soft total = 13-15 and dealer has 5-6, double
        elif (current_total == 13 or current_total == 14) and (dealer_first_card == 5 or dealer_first_card == 6):
            current_total = double_down(current_total)
        #if soft total = 13-15 and dealer has <=4 or >= 7, hit
        elif (current_total == 13 or current_total == 14) and (dealer_first_card <= 4 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        else:
            player_total = 70
            hit = False
            soft = False
    
        #######################
        #exit soft totals loop if gone over 21. Re-enter hardtotals loop
        if current_total > 21:
            current_total = current_total - 10
            soft = False
            #break
            #hit = False

    else: #need the else statement (in line with while loop because player_total is a function output and may not be defined in the if/elif statments, meaning the function can't return (for example when hitting) anything for this variable
        player_total = current_total #player total can equal current total with no harm done up until comparing to dealer's hand. By then all possible hands would have defined player_total as final value (written explicitly after stay and double function)
    return [player_total, current_total] #need to return player_total & current total because some hands continue after this loop and some stop, so both are important

def hard_totals(current_total, dealer_first_card):
    global soft #don't need soft here
    global hit
    global hit_count
    #global player_total
    while hit == True:
        #if hard total >=17, stay (also bust)
        if current_total >= 17:
            current_total = stay(current_total)
        #if hard total is 13-16 dealer <= 6, stay
        elif current_total <= 16 and current_total >= 13 and dealer_first_card <= 6:
            current_total = stay(current_total)
        #if hard total is 13-16 dealer > 6, hit
        elif current_total <= 16 and current_total >= 13 and dealer_first_card > 6:
            current_total = hit_me(current_total)
        #if hard total is 12 dealer = 2,3, >= 7, hit
        elif current_total == 12 and (dealer_first_card <= 3 or dealer_first_card >= 7):
            current_total = hit_me(current_total)
        #if hard total is 12 dealer = 4,5,6, stay
        elif current_total == 12 and (dealer_first_card >= 4 and dealer_first_card <= 6):
            current_total = stay(current_total)
        #if hard total = 11, doublz
        elif current_total == 11:
            current_total = double_down(current_total)
        #if hard total = 10 & dealer <=9, double
        elif current_total == 10 and dealer_first_card <= 9:
            current_total = double_down(current_total)
        #if hard total = 10 & dealer >9, hit
        elif current_total == 10 and dealer_first_card > 9:
            current_total = hit_me(current_total)
        #if hard total = 9 & dealer 3-6, double
        elif current_total == 9 and (dealer_first_card >= 3 and dealer_first_card <= 6):
            current_total= double_down(current_total)
        #if hard total = 9 & dealer =2 or >=7, hit
        elif current_total == 9 and (dealer_first_card == 2 or dealer_first_card >=7):
            current_total = hit_me(current_total)
        #if hard total <=8, hit
        elif current_total <= 8: 
            current_total = hit_me(current_total)
        else:
            player_total = 60
            hit = False
    else: #need the else statement (in line with while loop because player_total is a function output and may not be defined in the if/elif statments, meaning the function can't return (for example when hitting) anything for this variable
          player_total = current_total #player total can equal current total with no harm done up until comparing to dealer's hand. By then all possible hands would have defined player_total as final value (written explicitly after stay and double function)
    return player_total

def splits(current_total, first_card, dealer_first_card):
    global split
    global hit
    global soft
    global hit_count
    global bet
    global blackjack
    global double
    #define when to split and not
    if (first_card == 2 or first_card == 3) and dealer_first_card <=7:
        split = True
    elif (first_card == 2 or first_card == 3) and dealer_first_card > 7:
        split = False
    elif (first_card == 4) and (dealer_first_card == 5 or dealer_first_card == 6):
        split = True
    elif (first_card == 4) and (dealer_first_card <= 4 or dealer_first_card >= 7):
        split = False
    elif first_card == 5 or first_card == 10:
        split = False
    elif first_card == 6 and dealer_first_card <=6:
        split = True
    elif first_card == 6 and dealer_first_card >=7:
        split = False
    elif first_card == 7 and dealer_first_card <=7:
        split = True
    elif first_card == 7 and dealer_first_card >=8:
        split = False
    elif first_card == 8 or first_card == 11:
        split = True
    elif first_card == 9 and (dealer_first_card == 7 or dealer_first_card >=10):
        split = False
    elif first_card == 9 and (dealer_first_card <=6 or dealer_first_card == 8 or dealer_first_card == 9):
        split = True
    else:
        split = False
        current_total = 40
        player_total = 40

    #if not splitting, return to hard totals loop
    if split == False:
        player_total = hard_totals(current_total, dealer_first_card)
        double_list = []
        double_list.append(double)
        
        
    #If splitting, model the 2 hands seperatly
    elif split == True:
        player_total = [] #list to append totals to
        double_list = []
        n_hands = ["hand 1", "hand 2"]

        if random.choice(card_options) == 2*first_card:  #model 3rd split  by adding item to n_hands list. Has to be before for loop, so this just models whether a 3rd card will be the same as current
                n_hands.append("hand 3")
                if random.choice(card_options) == 2*first_card:  #model 3rd split
                    n_hands.append("hand 4")

        for hand in range(len(n_hands)):  #loop though and model each split hand a # of times equal to the length of n_hands list

            hand_total = hit_me(first_card) #first card is current total. This gives the hand 2 cards 
            hit_count = 0 #reset hit_count for each hand
            hit = True #reset initial variables or else hit=False for next hand
            soft=True
            bet = 5
            double = False 

            #if getting blackjack after splitting
            if (hand_total - first_card) == 10 and first_card == 11: #never split 10's, qst card would be and ace
                hand_total = 21
                bet = blackjack*bet

            #IF 2ND CARD IS AN ACE, RUN SOFTOTALS->HARDTOTALS, IF NOT AN ACE RUN HARDTOTALS
            if (hand_total - first_card) == 11:  
                soft_function = soft_totals(hand_total, dealer_first_card)
                hand_total = soft_function[0] #redefine function outputs because they're a list and global variables doens't work here
                hand_total = soft_function[1] #same as above

                if hit != False:  #only run hardtotals function if you didn't stay or double in soft total loop (you hit and went over 21)
                    hand_total = hard_totals(hand_total, dealer_first_card)

            else:
                hand_total = hard_totals(hand_total, dealer_first_card)

            double_list.append(double)
            player_total.append(hand_total)
            
    else:
        current_total = 87
        player_total = 87

    return [player_total, double_list]

In [5]:
#BLACKJACK MODELING WITH FUNCTIONS
#define input varibles
current_money = 15000
bet = 5
double = 2 #when doubling down
blackjack = 1.5

#define lists to later append data in
money = []
player_total_list = []
dealer_total_list = []
player_1st_list = []
player_2nd_list = []
dealer_1st_list = []
dealer_2nd_list = []
hit_count_list = []
outcome_list = []
double_list = []
split_list = []
thrid_card_list = []
forth_card_list = []
fifth_card_list = []
sixth_card_list = []

#start for loop to model 100,000 hands of play
for hand in range(1000000):

    #deal cards from infinite deck
    first_card = deals = random.choice(card_options)
    second_card = deals = random.choice(card_options)
    dealer_first_card = deals = random.choice(card_options)
    dealer_second_card = deals = random.choice(card_options)
    current_total = (first_card) + (second_card)
    
    hit = True #go through while loop at least once
    soft = True #go through softtotal while loop at least once if one card is an ace. loop through soft total until staying or going over 21 which exits this loop and goes to 1st loop for hard totals
    double = False #only for analysis to tell weather hand was doubled or not
    split = False #don't compare player's & dealer's hand unless cards were split
    player_total = 0
    dealer_total = 0
    bet = 5
    hit_count = 0


    ###########################################################################################################################################
    #model gampeplay
    ###########################################################################################################################################

    #blackjack
    if (first_card + second_card) == 21:
        player_total = 21
        hit = False
    ###########################################################################################################################################
    #if cards are pairs, SPLIT OR NOT
    elif (first_card) == (second_card):
        split_cards = splits(current_total, first_card, dealer_first_card)
        player_total = split_cards[0]
        double = split_cards[1]
    ###########################################################################################################################################

    ###########################################################################################################################################
    #SOFT TOTALS  -- if 1 card is an ace     
    elif (first_card) == 11 or (second_card) == 11:  #need the soft == True clause to avoid exiting softtotal loop, going through hardtotal loop, and coming back into softotal loop
        
        #run soft_totals function
        soft_function = soft_totals(current_total, dealer_first_card)

        #redefine function outputs because they're a list and global variables doens't work here
        player_total = soft_function[0]
        current_total = soft_function[1]

        #only run hardtotals function if you didn't stay or double in soft total loop (you hit and went over 21)
        if hit != False:  
            player_total = hard_totals(current_total, dealer_first_card)
    #########################################################################################################################################
    else:
        player_total = hard_totals(current_total, dealer_first_card)

    #model dealer's hand
    dealer_total = dealer_first_card + dealer_second_card
    while dealer_total < 17:
        dealer_total = hit_me(dealer_total, player = "False")

    #compare hands and append values to lists
    if split == True: #only run this if cards were split, have to be modeled as seperate hands
        for player_total in player_total:
            hand_results = compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card)
            current_money = hand_results[0]
            outcome = hand_results[1]

            #append variables to list
            money.append(current_money)
            player_total_list.append(player_total)
            dealer_total_list.append(dealer_total)
            player_1st_list.append(first_card)
            player_2nd_list.append(second_card)
            dealer_1st_list.append(dealer_first_card)
            dealer_2nd_list.append(dealer_second_card)
            hit_count_list.append(hit_count)
            outcome_list.append(outcome)
            split_list.append(split)
        for double in double:
            double_list.append(double)

    else: #if no split occurred
            hand_results = compare_hands(player_total, dealer_total, first_card, second_card, dealer_first_card, dealer_second_card)
            current_money = hand_results[0]
            outcome = hand_results[1]

            #append variables to list
            money.append(current_money)
            player_total_list.append(player_total)
            dealer_total_list.append(dealer_total)
            player_1st_list.append(first_card)
            player_2nd_list.append(second_card)
            dealer_1st_list.append(dealer_first_card)
            dealer_2nd_list.append(dealer_second_card)
            hit_count_list.append(hit_count)
            outcome_list.append(outcome)
            split_list.append(split)
            double_list.append(double)
            # thrid_card_list = []
            # forth_card_list = []
            # fifth_card_list = []
            # sixth_card_list.append()

    #reset variables
    player_total = 0
    dealer_total = 0
    bet = 5
    hit_count = 0
  


In [6]:
#CREATE DATAFRAME
df = pd.DataFrame({"Current Money":money,
                   "Player Totals": player_total_list,
                   "Dealer Totals": dealer_total_list,
                   "Player 1st Card": player_1st_list,
                   "Player 2nd Card": player_2nd_list,
                   "Dealer 1st Card": dealer_1st_list,
                   "Dealer 2nd Card": dealer_2nd_list,
                   "Hit Count": hit_count_list,
                   "Outcome": outcome_list,
                   "Double": double_list,
                   "Split": split_list})
df["Hit Count"].value_counts()

Hit Count
0    483207
1    408837
2    110912
3     21300
4      2677
5       216
6        14
Name: count, dtype: int64

In [7]:
#VALUE COUNTS OF PLAYER TOTALS
df["Player Totals"].value_counts() # 10 and 11 can still come up if doubling on 9 or 10 and you get a 2

Player Totals
20    166673
19    122602
17    122577
18    116798
21    108870
13     50487
22     48770
15     45258
14     44630
16     41003
23     39529
24     34212
12     32175
25     29655
26     22991
11       933
Name: count, dtype: int64

In [8]:
#OVERALL OUTCOME PERCENTAGES
win_percent = len(df.loc[df["Outcome"] == "Win"])/len(df)
dealer_bust_percent = len(df.loc[df["Outcome"] == "Dealer Bust"])/len(df)
loss_percent = len(df.loc[df["Outcome"] == "Loss"])/len(df)
push_percent = len(df.loc[df["Outcome"] == "Push"])/len(df)
blackjack_percent = len(df.loc[df["Outcome"] == "Blackjack"])/len(df)
bust_percent = len(df.loc[df["Outcome"] == "Bust"])/len(df)

print(f"Win %: {win_percent:.2%}")
print(f"Dealer Bust %: {dealer_bust_percent:.2%}")
print(f"Loss % %: {loss_percent:.2%}")
print(f"Push %: {push_percent:.2%}")
print(f"Blackjack %: {blackjack_percent:.2%}")
print(f"Bust %: {bust_percent:.2%}")

Win %: 15.33%
Dealer Bust %: 24.56%
Loss % %: 29.89%
Push %: 8.79%
Blackjack %: 4.38%
Bust %: 17.05%


In [9]:
#CREATE HAND ID FOR ALL HAND COMBINATIONS TO GROUPBY
df["Hand1"] = df[["Player 2nd Card", "Player 1st Card"]].min(axis=1) #need "axis=1" for some reason
df["Hand2"] = df[["Player 2nd Card", "Player 1st Card"]].max(axis=1) #need "axis=1" for some reason
df = df.sort_values(by= "Hand1", ascending=True)
df["Hand (1st,2nd / Dealer's)"] = df["Hand1"].astype(str) + "," + df["Hand2"].astype(str) + " / " + df["Dealer 1st Card"].astype(str)
df["Player Hand"] = df["Hand1"].astype(str) + "," + df["Hand2"].astype(str)
df = df.drop(columns=["Hand2", "Hand1"])
df.head(5)



,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Outcome,Double,Split,"Hand (1st,2nd / Dealer's)",Player Hand
0,14990.0,12,21,2,9,5,5,1,Loss,True,False,"2,9 / 5","2,9"
430677,45957.5,25,22,2,5,7,5,2,Bust,False,False,"2,5 / 7","2,5"
430679,45952.5,18,18,4,2,7,11,2,Push,False,False,"2,4 / 7","2,4"
430693,45970.0,15,17,2,9,6,6,1,Loss,True,False,"2,9 / 6","2,9"
430702,45985.0,14,25,2,9,6,4,1,Dealer Bust,True,False,"2,9 / 6","2,9"


In [10]:
#SEPERATE OUTCOMES INTO NEW COLUMNS
win_only = []
overall_win = []
loss_only = []
overall_loss = []
bust = []
dealer_bust = []
blackjack = []
push = []

#run for loop through outcome column to seperate outcomes into new column so I can get a % of each outcome by hand
for outcome in df["Outcome"]:
    if outcome == "Win":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(outcome)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Loss":
        overall_win.append(np.nan)
        overall_loss.append(outcome)
        win_only.append(np.nan)
        loss_only.append(outcome)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Bust":
        overall_win.append(np.nan)
        overall_loss.append(outcome)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(outcome)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Dealer Bust":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(outcome)
        blackjack.append(np.nan)
        push.append(np.nan)
    elif outcome == "Blackjack":
        overall_win.append(outcome)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(outcome)
        push.append(np.nan)
    elif outcome == "Push":
        overall_win.append(np.nan)
        overall_loss.append(np.nan)
        win_only.append(np.nan)
        loss_only.append(np.nan)
        bust.append(np.nan)
        dealer_bust.append(np.nan)
        blackjack.append(np.nan)
        push.append(outcome)

df["Overall Win"] = overall_win
df["Overall Loss"] = overall_loss
df["Win Only"] = win_only
df["Loss Only"] = loss_only
df["Bust"] = bust
df["Dealer Bust"] = dealer_bust
df["Blackjack"] = blackjack
df["Push"] = push
df.head(5)

,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Outcome,Double,...,"Hand (1st,2nd / Dealer's)",Player Hand,Overall Win,Overall Loss,Win Only,Loss Only,Bust,Dealer Bust,Blackjack,Push
0,14990.0,12,21,2,9,5,5,1,Loss,True,...,"2,9 / 5","2,9",NaN,Loss,NaN,Loss,NaN,NaN,NaN,NaN
430677,45957.5,25,22,2,5,7,5,2,Bust,False,...,"2,5 / 7","2,5",NaN,Bust,NaN,NaN,Bust,NaN,NaN,NaN
430679,45952.5,18,18,4,2,7,11,2,Push,False,...,"2,4 / 7","2,4",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Push
430693,45970.0,15,17,2,9,6,6,1,Loss,True,...,"2,9 / 6","2,9",NaN,Loss,NaN,Loss,NaN,NaN,NaN,NaN
430702,45985.0,14,25,2,9,6,4,1,Dealer Bust,True,...,"2,9 / 6","2,9",Dealer Bust,NaN,NaN,NaN,NaN,Dealer Bust,NaN,NaN


In [11]:
#GROUPBY PLAYER HAND COMBINATIONS
groupby_hand = df.groupby(["Hand (1st,2nd / Dealer's)"]).count()
groupby_hand["Win Only %"] = (groupby_dealer_hand["Win Only"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)  #I can divide the win count by any column because I aggregated by count when grouping by, so all other columns are the same. Same with the 5 lines below
groupby_hand["Overall Win %"] = (groupby_hand["Overall Win"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Overall Loss %"] = (groupby_hand["Overall Loss"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Dealer Bust %"] = (groupby_hand["Dealer Bust"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Loss Only %"] = (groupby_hand["Loss Only"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Bust %"] = (groupby_hand["Bust"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Blackjack %"] = (groupby_hand["Blackjack"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand["Push %"] = (groupby_hand["Push"]/groupby_hand["Outcome"]).map('{:,.2%}'.format)
groupby_hand.reset_index().head(5)

,"Hand (1st,2nd / Dealer's)",Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 1st Card,Dealer 2nd Card,Hit Count,Outcome,...,Blackjack,Push,Win Only %,Overall Win %,Overall Loss %,Dealer Bust %,Loss Only %,Bust %,Blackjack %,Push %
0,"10,10 / 10",29238,29238,29238,29238,29238,29238,29238,29238,29238,...,0,10029,33.31%,54.43%,11.27%,21.12%,11.27%,0.00%,0.00%,34.30%
1,"10,10 / 11",7228,7228,7228,7228,7228,7228,7228,7228,7228,...,0,754,31.17%,56.41%,33.16%,25.24%,33.16%,0.00%,0.00%,10.43%
2,"10,10 / 2",7265,7265,7265,7265,7265,7265,7265,7265,7265,...,0,821,38.95%,77.43%,11.27%,38.47%,11.27%,0.00%,0.00%,11.30%
3,"10,10 / 3",7261,7261,7261,7261,7261,7261,7261,7261,7261,...,0,849,37.86%,77.22%,11.09%,39.36%,11.09%,0.00%,0.00%,11.69%
4,"10,10 / 4",7307,7307,7307,7307,7307,7307,7307,7307,7307,...,0,830,36.80%,77.75%,10.89%,40.95%,10.89%,0.00%,0.00%,11.36%


In [17]:
#GROUPBY DEALER HAND COMBINATIONS
groupby_dealer_hand = df.groupby(["Dealer 1st Card"]).count()
groupby_dealer_hand["Win Only %"] = (groupby_dealer_hand["Win Only"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)  #I can divide the win count by any column because I aggregated by count when grouping by, so all other columns are the same. Same with the 5 lines below
groupby_dealer_hand["Overall Win %"] = (groupby_dealer_hand["Overall Win"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Overall Loss %"] = (groupby_dealer_hand["Overall Loss"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Dealer Bust %"] = (groupby_dealer_hand["Dealer Bust"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Loss Only %"] = (groupby_dealer_hand["Loss Only"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Bust %"] = (groupby_dealer_hand["Bust"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Blackjack %"] = (groupby_dealer_hand["Blackjack"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand["Push %"] = (groupby_dealer_hand["Push"]/groupby_dealer_hand["Outcome"]).map('{:,.2%}'.format)
groupby_dealer_hand.reset_index().head(5)

,Dealer 1st Card,Current Money,Player Totals,Dealer Totals,Player 1st Card,Player 2nd Card,Dealer 2nd Card,Hit Count,Outcome,Double,...,Blackjack,Push,Win Only %,Overall Win %,Overall Loss %,Dealer Bust %,Loss Only %,Bust %,Blackjack %,Push %
0,2,80283,80283,80283,80283,80283,80283,80283,80283,80283,...,3664,4682,11.42%,50.67%,43.50%,34.69%,40.30%,3.20%,4.56%,5.83%
1,3,80135,80135,80135,80135,80135,80135,80135,80135,80135,...,3572,4634,10.94%,52.09%,42.13%,36.70%,39.09%,3.04%,4.46%,5.78%
2,4,80193,80193,80193,80193,80193,80193,80193,80193,80193,...,3601,4132,9.63%,53.76%,41.09%,39.64%,41.09%,0.00%,4.49%,5.15%
3,5,80616,80616,80616,80616,80616,80616,80616,80616,80616,...,3627,3953,9.31%,55.29%,39.81%,41.49%,39.81%,0.00%,4.50%,4.90%
4,6,80697,80697,80697,80697,80697,80697,80697,80697,80697,...,3723,4248,10.39%,54.92%,39.82%,39.92%,39.82%,0.00%,4.61%,5.26%


In [ ]:
len(groupby_hand)

In [19]:
groupby_hand.to_csv("blackjack_results.csv")
groupby_dealer_hand.to_csv("blackjack_groupby_dealer.csv")

In [ ]:
#AVG PLAYER EDGE (loss/hand)
(df["Current Money"][99999] - df["Current Money"][0])/1000/5

In [ ]:
# #GRAPH MONEY OVER TIME
# mychart = df["Current Money"].plot(kind='line')
# mychart.set_xlabel("# of hands")
# mychart.set_ylabel("Money")